In [1]:
import numpy as np
import pandas as pd
from math import sqrt

from mlgrad.cluster import KMeans, KMeansMahalanobis

In [2]:
def permutate_values(Yp, Y, m):
    d = {}
    for yp,y in zip(Yp, Y):
        c = d.setdefault((yp,y), 0)
        d[(yp,y)] = c+1
    print(d)
    values = np.array(list(d.values()))
    keys = list(d.keys())
    indices = list(np.argsort(values))
    keys = [keys[indices[-i]] for i in range(1,m+1)]
    print(keys)
    d = dict(keys)
    for i in range(len(Y)):
        yp = Yp[i]
        Yp[i] = d[yp]

In [4]:
sdata = pd.read_csv("data/cbb/s2.txt", names=['x1', 'x2'], sep='\s+')

In [6]:
X = np.c_[sdata['x1'], sdata['x2']]
X = np.ascontiguousarray(X, 'd')
# X = scale(X)  
print(len(X))
print(X)

5000
[[845753. 636607.]
 [812954. 643720.]
 [868217. 609046.]
 ...
 [706472. 171207.]
 [659128. 142611.]
 [665898.  98088.]]


In [7]:
x1_min, x1_max = X[:,0].min(), X[:,0].max()
x2_min, x2_max = X[:,1].min(), X[:,1].max()
x_min = min(x1_min, x2_min)
x_max = max(x1_max, x2_max)
extent = (x_min, x_max, x_min, x_max)
xrange1 = np.linspace(x_min, x_max, 100)
xrange2 = np.linspace(x_min, x_max, 100)
XX1, XX2, = np.meshgrid(xrange1, xrange2)
XX = np.c_[XX1.ravel(), XX2.ravel()]

In [8]:
plt.figure(figsize=(6,5))
plt.scatter(X[:,0], X[:,1], c='k', edgecolors='w', marker='s', s=9)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
alpha = 0.9

In [ ]:
# avgmin = avragg.SoftMinimal(2.0)
avgmin = avragg.Minimal()
avrfunc = averager_it(func.QuantileFunc(alpha, func.Sqrt(0.001)))
# avrfunc = averager_it(func.QuantileFunc(alpha, func.Power(1.01, 0.001)))
avrfunc = avragg.ParameterizedAverage(func.WinsorizedSmoothFunc(func.Sqrt(0.001)), avrfunc)
hcd = mlocation_scatter.MLocationsScattersEstimator(avrfunc, avgmin, 15, 
                                    tol=1.0e-9, n_iter=200)

In [ ]:
hcd.fit(X)

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(hcd.dvals)
plt.subplot(1,2,2)
plt.plot(hcd.dvals2)
plt.show()

In [ ]:
D = hcd.evaluate_dist(XX)
DD = D.reshape(XX1.shape)
print(D.min(), D.max())

In [ ]:
# plt.figure(figsize=(8,7))
# plt.scatter(X[:,0], X[:,1], c='k', edgecolors='w', marker='s', s=9)
# plt.contour(DD, origin='lower', extent=extent, 
#             levels=np.linspace(0.0,D.max(),100), colors='grey', linewidths=0.5)
# plt.scatter(hcd.locs[:,0], hcd.locs[:,1], c='k', edgecolors='k', s=100)
# plt.show()

In [ ]:
# avgmin = avragg.SoftMinimal(2.0)
avgmin2 = avragg.Minimal()
avrfunc2 = averager_it(func.QuantileFunc(alpha, func.Sqrt(0.001)))
# avrfunc = averager_it(func.QuantileFunc(alpha, func.Power(1.01, 0.001)))
avrfunc2 = avragg.ParameterizedAverage(func.WinsorizedSmoothFunc(func.Sqrt(0.001)), avrfunc2)
hcd2 = mlocation_scatter.MLocationsScattersEstimator(avrfunc2, avgmin2, 15, 
                                    tol=1.0e-9, n_iter=200)

In [ ]:
hcd2.fit(X)

In [ ]:
D_2 = hcd2.evaluate_dist(XX)
DD_2 = D_2.reshape(XX1.shape)
print(D_2.min(), D_2.max())

### Среднее арифметическое

In [ ]:
arith = avragg.ArithMean()
km = mlocation_scatter.MLocationsScattersEstimator(arith, avgmin, 15, 
                                    tol=1.0e-9, n_iter=200)

In [ ]:
km.fit(X)

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(km.dvals)
plt.subplot(1,2,2)
plt.plot(km.dvals2)
plt.show()

In [ ]:
D2 = km.evaluate_dist(XX)
DD2 = D2.reshape(XX1.shape)
print(D2.min(), D2.max())

In [ ]:
# plt.figure(figsize=(7,6))
# plt.contour(DD2, origin='lower', extent=extent, 
#             levels=np.linspace(0.,D2.max(),100), colors='grey', linewidths=0.5)
# plt.scatter(X[:,0], X[:,1], c='k', edgecolors='w', marker='s', s=9)
# plt.scatter(km.locs[:,0], km.locs[:,1], c='k', edgecolors='w', s=100)
# plt.show()

In [ ]:
arith2 = avragg.ArithMean()
km2 = mlocation_scatter.MLocationsScattersEstimator(arith2, avgmin, 15, 
                                    tol=1.0e-9, n_iter=200)

In [ ]:
km2.fit(X)

In [ ]:
D2_2 = km2.evaluate_dist(XX)
DD2_2 = D2_2.reshape(XX1.shape)
print(D2_2.min(), D2_2.max())

In [ ]:
plt.figure(figsize=(14,6.5))

plt.subplot(1,2,1)
plt.title('Robust algorithm')
plt.contour(DD, origin='lower', extent=extent, 
            levels=np.linspace(0.,D.max(),100), colors='grey', linewidths=0.5)
plt.scatter(X[:,0], X[:,1], c='w', edgecolors='k', marker='o', linewidths=0.5, s=16)
plt.scatter(hcd.locs[:,0], hcd.locs[:,1], c='k', edgecolors='k', s=100)

plt.subplot(1,2,2)
plt.title('Regular algorithm')
plt.contour(DD2, origin='lower', extent=extent, 
            levels=np.linspace(0.,D2.max(),100), colors='grey', linewidths=0.5)
plt.scatter(X[:,0], X[:,1], c='w', edgecolors='k', marker='o', linewidths=0.5, s=16)
plt.scatter(km.locs[:,0], km.locs[:,1], c='k', edgecolors='w', s=100)
plt.savefig('fig/robust_kmeans_center_variance_s4.eps')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14,6.5))

plt.subplot(1,2,1)
plt.title('Robust algorithm')
plt.contour(DD_2, origin='lower', extent=extent, 
            levels=np.linspace(0.,D_2.max(),50), colors='grey', linewidths=0.5)
plt.scatter(X[:,0], X[:,1], c='k', edgecolors='w', marker='s', s=9)
plt.scatter(hcd2.locs[:,0], hcd2.locs[:,1], c='k', edgecolors='k', s=100)

plt.subplot(1,2,2)
plt.title('Regular algorithm')
plt.contour(DD2_2, origin='lower', extent=extent, 
            levels=np.linspace(0.,D2_2.max(),50), colors='grey', linewidths=0.5)
plt.scatter(X[:,0], X[:,1], c='k', edgecolors='w', marker='s', s=9)
plt.scatter(km2.locs[:,0], km2.locs[:,1], c='k', edgecolors='w', s=100)
plt.savefig('fig/robust_kmeans_center_variance2_s4.eps')
plt.tight_layout()
plt.show()

In [ ]:
D_1 = hcd.evaluate_dist(X)
D_2 = km.evaluate_dist(X)
D2_1 = hcd2.evaluate_dist(X)
D2_2 = km2.evaluate_dist(X)
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
# plt.plot(sorted(D_1), label='robust (S)', c='k', marker='s', markersize=4)
# plt.plot(sorted(D_2), label='regular (S)', c='k', linestyle='--', marker='s', markersize=4)
plt.scatter(D_2, D_1)
plt.xlabel('regular')
plt.ylabel('robust')
plt.legend()
# plt.ylim(1e-6, 5)
plt.grid(1)
plt.subplot(1,2,2)
plt.plot(sorted(D2_1), label='robust', c='k')
plt.plot(sorted(D2_2), label='regular', c='k', linestyle='--')
plt.legend()
# plt.ylim(1e-6, 5)
plt.grid(1)
plt.savefig('fig/distances_s4.eps')
plt.show()